In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

df_X = pd.read_csv('Xsvm.csv',header=None)
df_Y = pd.read_csv('ysvm.csv',header=None)
X = np.array(df_X,dtype=np.float64)
Y = np.array(df_Y,dtype=np.float64)
print(X.shape,Y.shape)

(500, 2) (500, 1)


In [2]:
# Convex Optimization
a = cp.Variable(len(Y))
R1 = cp.matmul(cp.diag(a),Y)
R2 = cp.matmul(X.T,R1)
R4 = cp.norm(R2)**2
R4.shape

()

In [3]:
P1 = cp.sum(a)
Const1 = P1 - 0.5*R4
# Const1 = np.reshape(Const1,(1,))

In [4]:
Const2 = cp.matmul(a.T,Y)
Const3 = [0<=a,Const2 == 0]
obj = cp.Maximize(Const1)
prob = cp.Problem(obj, Const3)
prob.solve(verbose=True)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.047e+00  +2e+03  1e+00  8e-01  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  -7.040e+02  -2.216e+02  +1e+03  8e+01  6e+01  6e+02  2e+00  0.4946  4e-01   1  2  2 |  0  0
 2  -2.717e+01  -2.927e+00  +8e+02  6e-01  4e-01  3e+01  2e+00  0.9890  7e-01   2  2  2 |  0  0
 3  -4.351e+01  -7.816e+00  +4e+02  5e-01  3e-01  4e+01  8e-01  0.6145  2e-01   2  2  2 |  0  0
 4  -7.101e+01  -3.516e+01  +2e+02  1e+00  3e-01  4e+01  4e-01  0.8705  4e-01   2  1  1 |  0  0
 5  -2.276e+01  -2.158e+01  +2e+02  9e-02  2e-01  2e+00  3e-01  0.5327  7e-01   2  2  1 |  0  0
 6  -6.649e+01  -6.505e+01  +5e+01  6e-02  7e-02  2e+00  1e-01  0.9890  3e-01   2  1  1 |  0  0
 7  -6.052e+01  -6.015e+01  +1e+01  1e-02  1e-02  4e-01  2e-02  0.8228  4e-02   3  1  1 |  0  0
 8  -6.507e+01  -6.494e+01  +5e+00  3e-03  7e-

69.4370298544791

In [5]:
print(a.value)

[ 8.51018788e-10 -2.71507558e-11 -2.70288781e-11 -2.35464522e-11
  5.47346537e-10  1.30634550e-10 -1.92538422e-11  9.17407642e-11
 -1.23736851e-11  5.30435081e-11  1.16139868e-10  5.50013584e-11
  5.13190261e-11 -2.56644585e-11  6.71756136e-11  1.43589355e-11
 -2.09576866e-11  7.24115193e-11  1.67802591e-11  1.58583485e-10
  3.38803624e-11  1.92931528e-11  8.09015445e-11  3.58579995e-11
 -1.31398285e-11  4.92178023e-11 -7.76960976e-13 -2.64125177e-11
  1.09997538e-10 -2.80872080e-11  1.16793801e-10  9.55092905e-11
  5.71009004e-12  3.53511099e+00  2.58489136e-10  1.19245521e-10
 -1.78703611e-11 -1.66364058e-11 -2.73026781e-11 -2.11939454e-12
 -3.21948468e-11  6.66526414e-10  3.64173742e-11  1.59505648e-11
  7.02568473e-11  3.78274647e-12  3.52780990e-10  4.33688595e-11
 -4.47497882e-12 -7.55990574e-12  8.77875462e-11  8.80406680e-10
  1.55488270e-10  3.57707272e-10 -3.17151837e-11  1.38397546e-09
  6.25457641e-13  1.50537585e-10 -1.25260898e-11  2.17724987e-10
  8.86444481e-10  1.68793

In [6]:
A = (np.array(a.value)).reshape(500,1)

In [7]:
W = np.zeros((2,))
for i in range(len(Y)):
    W += A[i]*Y[i]*(X[i].T)
    if(A[i]>1e-4):
        print(i)

33
281
469


In [8]:
print(W)

[-0.11843836 11.78388371]


In [9]:
W0 = (1/Y[281]) - np.dot(W,X[281])
print(W0)

[0.10229773]


In [10]:
Test = np.array([[1.9, 0.4],[0.9, 0.9],[1.4, 1.5],[0.01, 0.005]])

for i in range(len(Test)):
    est = np.sign(np.dot(W,Test[i])+W0)
    print(Test[i],est)

[1.9 0.4] [1.]
[0.9 0.9] [1.]
[1.4 1.5] [1.]
[0.01  0.005] [1.]


In [11]:
# Verification
from sklearn import svm

clf = svm.SVC()
clf.fit(X,Y)

/home/grumptitan/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/grumptitan/.local/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [12]:
clf.predict(Test)

array([1., 1., 1., 1.])